In [10]:
import requests
import pandas as pd
import json

In [17]:
r=requests.get('https://run.opencravat.org/submit/annotate?chrom=chr1&pos=2406483&ref_base=C&alt_base=G&annotators=aloft,cadd,cadd_exome,cancer_genome_interpreter,ccre_screen,chasmplus,civic,clingen,clinpred,clinvar,cosmic,cosmic_gene,cscape,dann,dann_coding,dbscsnv,dbsnp,dgi,ensembl_regulatory_build,ess_gene,exac_gene,fathmm,fathmm_xf_coding,funseq2,genehancer,gerp,ghis,gnomad,gnomad3,gnomad_gene,gtex,gwas_catalog,linsight,loftool,lrt,mavedb,metalr,metasvm,mutation_assessor,mutationtaster,mutpred1,mutpred_indel,ncbigene,ndex,ndex_chd,ndex_signor,omim,pangalodb,phastcons,phdsnpg,phi,phylop,polyphen2,prec,provean,repeat,revel,rvis,segway,sift,siphy,spliceai,uniprot,vest,cgc,cgd,varity_r&reports=vcf')


In [18]:
r.json()

{'revel': None,
 'cadd_exome': None,
 'cancer_genome_interpreter': None,
 'ccre_screen': None,
 'cgc': None,
 'cgd': None,
 'chasmplus': None,
 'varity_r': None,
 'clingen': None,
 'clinpred': None,
 'clinvar': {'sig': 'Likely pathogenic',
  'disease_refs': 'MONDO:MONDO:0013936,MedGen:C3553947,OMIM:614870|MONDO:MONDO:0013937,MedGen:C3553948,OMIM:614871|MedGen:CN517202',
  'disease_names': 'Peroxisome biogenesis disorder 6A (Zellweger)|Peroxisome biogenesis disorder 6B|not provided',
  'rev_stat': 'criteria provided, single submitter',
  'id': 554342,
  'sig_conf': None},
 'cosmic': None,
 'cosmic_gene': {'gene_count': [['large intestine', '29'],
   ['skin', '22'],
   ['liver', '21'],
   ['stomach', '13'],
   ['lung', '12'],
   ['haematopoietic and lymphoid tissue', '11'],
   ['endometrium', '10'],
   ['breast', '9'],
   ['prostate', '9'],
   ['central nervous system', '8'],
   ['pancreas', '7'],
   ['upper aerodigestive tract', '6'],
   ['oesophagus', '5'],
   ['kidney', '4'],
   ['thy

In [5]:
len(r.json().keys())

68

In [8]:
# Helper function to flatten the nested dictionaries
def flatten_dict(d, parent_key='', sep='.'):
    items = {}
    for key, value in d.items():
        new_key = f'{parent_key}{sep}{key}' if parent_key else key
        if isinstance(value, dict):
            items.update(flatten_dict(value, new_key, sep=sep))
        else:
            items[new_key] = value
    return items

# Flatten the dictionary
flat_data = flatten_dict(r.json())

In [9]:
# Create a DataFrame
df = pd.DataFrame.from_dict(flat_data, orient='index').transpose()
df.head()

,revel,cadd_exome,cancer_genome_interpreter,ccre_screen,cgc,cgd,chasmplus,varity_r,clingen,clinpred,...,crx.note_variant,crx.coding,crx.hugo,crx.transcript,crx.so,crx.exonno,crx.cchange,crx.achange,crx.all_mappings,crx.gposend
0,None,None,None,None,None,None,None,None,None,None,...,,,PEX10,ENST00000447513.6,splice_site_variant,6,c.912+1G>C,,"{""PEX10"": [[""O60683"", """", ""intron_variant,spli...",2406483


In [11]:
# Extract data from 'crx' dictionary
crx_data = r.json().get('crx', {})

# Parse 'all_mappings' JSON and extract ENST mappings
all_mappings_json = crx_data.get('all_mappings', '{}')
all_mappings = json.loads(all_mappings_json)

# Create a list of dictionaries where each dictionary represents an ENST mapping
enst_data = []
for enst, mappings in all_mappings.items():
    enst_dict = {'ENST': enst}
    for mapping in mappings:
        gene, _, _, transcript, variant = mapping
        enst_dict[f'{gene}.transcript'] = transcript
        enst_dict[f'{gene}.variant'] = variant
    enst_data.append(enst_dict)

# Create a DataFrame from the list of ENST mappings
df = pd.DataFrame(enst_data)

# Fill NaN values with None
df = df.fillna('None')
df.head()

,ENST,O60683.transcript,O60683.variant,.transcript,.variant,O15258.transcript,O15258.variant
0,PEX10,ENST00000447513.6,c.912+1G>C,ENST00000510434.1,c.*279G>C,None,None
1,RER1,None,None,ENST00000378513.7,c.*3496C>G,ENST00000605895.6,c.*3359C>G
